In [3]:
import cv2
import mediapipe as mp
import csv
import pandas as pd
import os
import numpy as np
import tensorflow as tf

In [4]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [28]:
#DATASET_DIR = 'Single_person_violent'
DATASET_DIR = 'Final_Dataset'
CLASSES_LIST = ["Idle","Block","Kicking","Punching"]
OUTPUT_DIR = 'Output'
custom_headers = ['Frame Number', 'x','y','z','Visibility']

In [4]:
# Convert mediapipe landmarks into proper format for storing into a CSV file
def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    #print(f"Landmark coordinates for frame {frame_number}:")
    for landmark in landmarks:
        #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, landmark.x, landmark.y, landmark.z, landmark.visibility])    

In [8]:
# Convert a video into landmarks using mediapipe
def convert_video_to_landmark_csv(video_path):
    frame_number = 0
    csv_data = []

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():            
            ret, frame = cap.read()
            if not ret:
                break
            frame_number += 1
            # Convert the image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Perform pose detection
            results = pose.process(image)

            # Convert the image back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw the pose annotation on the image
            # if results.pose_landmarks:
            #     mp_drawing.draw_landmarks(
            #         image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Add the landmark coordinates to the list and print them
            write_landmarks_to_csv(results.pose_landmarks.landmark, frame_number, csv_data)   
    cap.release() 

    # Extract file and class names from the video path
    file_name = os.path.splitext(os.path.basename(video_path))[0]
    class_name = os.path.basename(os.path.dirname(video_path))

    
    
    # Ensure the output directory exists
    output_file_dir = os.path.join(OUTPUT_DIR, class_name)        
    os.makedirs(output_file_dir, exist_ok=True)  

    output_file = os.path.join(output_file_dir, f'{file_name}.csv')

    with open(f'{output_file}', mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write each row of the 2D array
        for row in csv_data:
            writer.writerow(row)

In [6]:
# Convert all videos in the given CLASSES_LIST into landmark csv files
def create_dataset(CLASSES_LIST):
    print('List: ',CLASSES_LIST)
    for class_name in CLASSES_LIST:
        print(f"Extracting data from {class_name}")
        # Get list of videos for each class
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            convert_video_to_landmark_csv(video_file_path)

In [57]:
# Create a sequence from a CSV file for making a single dataframe
def createSequence(csv_path, label):
    # Read the CSV file into a DataFrame
    data = pd.read_csv(csv_path, header=None, names=custom_headers)
    
    # Initialize the sequence with the label
    sequence = [label]
    
    # Group the data by 'Frame Number' and collect frame data
    grouped = data.groupby('Frame Number')[['x', 'y', 'z', 'Visibility']].apply(lambda x: x.values.tolist())
    
    # Extend the sequence with the grouped frame data
    sequence.extend(grouped.tolist())
    
    return sequence


In [58]:
# Read data from CSV and append to processed_df
sequence = createSequence('output.csv','Kicking')
processed_df = pd.DataFrame([sequence])
#processed_df2 = processed_df.append(pd.DataFrame(frame, columns=['x', 'y', 'z', 'Visibility']), ignore_index=True)
processed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,Kicking,"[[0.4208270311355591, 0.2813926935195923, -0.5...","[[0.4226616024971008, 0.2830023467540741, -0.4...","[[0.422335147857666, 0.2832168936729431, -0.48...","[[0.4239648282527923, 0.2812423706054687, -0.2...","[[0.4259682595729828, 0.2790455818176269, -0.2...","[[0.424310564994812, 0.2764437794685364, -0.26...","[[0.424225777387619, 0.2755632400512695, 0.049...","[[0.4241669476032257, 0.2759043276309967, -0.1...","[[0.4190621674060821, 0.2758240699768066, -0.0...",...,"[[0.4902646839618683, 0.266271561384201, -0.52...","[[0.4924321174621582, 0.2656450271606445, -0.5...","[[0.4942595660686493, 0.2644723653793335, -0.5...","[[0.4932672381401062, 0.2639179527759552, -0.5...","[[0.4929601550102234, 0.264041006565094, -0.53...","[[0.4959222972393036, 0.2640158534049988, -0.5...","[[0.5004463791847229, 0.2640080451965332, -0.5...","[[0.5007750391960144, 0.2638412415981293, -0.4...","[[0.4997018575668335, 0.2627485990524292, -0.4...","[[0.4992328286170959, 0.262261152267456, -0.44..."


In [64]:
dat = pd.read_csv('output.csv',header=None,names=custom_headers)
dat['Frame Number'].tail(1).values[0]
#print(dat.groupby('FrameNumber')['x'].apply(lambda x: x.tolist()).iloc[:1][1])

37

In [7]:
# Get all the CSV files of the given CLASSES_LIST
def getAllCSV(CLASSES_LIST):
    files = {}
    for class_name in CLASSES_LIST:        
        # Get list of csv files for each class
        files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))
        for file_name in files_list:
            # Get the complete csv path.
            files.setdefault(class_name,[]).append(os.path.join(OUTPUT_DIR, class_name, file_name))
    return files

In [14]:
# Merge all the sequences into a single dataframe and return it
def createDataframe(CLASSES_LIST):
    df_array = []    
    for class_name in CLASSES_LIST:        
        # Get list of csv files for each class
        files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))
        for file_name in files_list:
            # Get the complete csv path.
            csv_file_path = os.path.join(OUTPUT_DIR, class_name, file_name)            
            df_array.append(createSequence(csv_file_path,class_name))
    return pd.DataFrame(df_array)
            

In [8]:
# Calculate the mean and standard deviation of z values (depth values)
def calculateZParamsForNormalization(CLASSES_LIST,csv_files):    
    z_values = []
    for key in CLASSES_LIST:
        for file in csv_files[key]:
            df = pd.read_csv(file,header=None,names=custom_headers)
            z_values.extend(df['z'].values)
    z_mean = np.mean(z_values)
    z_std = np.std(z_values)
    return z_mean, z_std

In [9]:
# Calculate the min and max of z values (depth values)
def calculateZMinMaxForNormalization(CLASSES_LIST,csv_files):    
    z_values = []
    for key in CLASSES_LIST:
        for file in csv_files[key]:
            df = pd.read_csv(file,header=None,names=custom_headers)
            z_values.extend(df['z'].values)
    z_min = np.min(z_values)
    z_max = np.max(z_values)
    return z_min, z_max

In [10]:
# Normalize all the z values (depth values) in the CSV files
def normalizeZValuesInCSV(CLASSES_LIST):    
    files = getAllCSV(CLASSES_LIST)
    z_min, z_max = calculateZMinMaxForNormalization(CLASSES_LIST,files)
    for key in files:
        for file in files[key]:
            df = pd.read_csv(file,header=None,names=custom_headers)
            # Normalize the 'z' column using Min-Max scaling
            df['z'] = (df['z'] - z_min) / (z_max - z_min)
            df.to_csv(file, index=False, header=False)

In [20]:
def findHighestFrameNumber(CLASSES_LIST):
    highest_frame_number = 0    
    files = getAllCSV(CLASSES_LIST)        
    highest_frame_video = ''
    for key in files:        
        for file in files[key]:            
            df = pd.read_csv(file,header=None,names=custom_headers)
            current_frame_number = df['FrameNumber'].tail(1).values[0]
            if  current_frame_number > highest_frame_number:
                highest_frame_number = current_frame_number  
                highest_frame_video  = file      
    return highest_frame_number,highest_frame_video

In [21]:
def findFramesHigherThanX(CLASSES_LIST,X):        
    files = getAllCSV(CLASSES_LIST)        
    high_frame_videos = []
    for key in files:        
        for file in files[key]:            
            df = pd.read_csv(file,header=None,names=custom_headers)
            current_frame_number = df['FrameNumber'].tail(1).values[0]
            if  current_frame_number > X:
                high_frame_videos.append((file,current_frame_number))
    return high_frame_videos

In [22]:
def deleteFramesHigherThanX(CLASSES_LIST,X):    
    custom_headers = ['FrameNumber','x','y','z','Visibility']
    files = getAllCSV(CLASSES_LIST)        
    for key in files:        
        for file in files[key]:            
            df = pd.read_csv(file,header=None,names=custom_headers)
            current_frame_number = df['FrameNumber'].tail(1).values[0]
            if  current_frame_number > X:
                os.remove(file)

In [9]:
create_dataset(CLASSES_LIST)

List:  ['Kicking', 'Punching', 'Block', 'Idle']
Extracting data from Kicking
Extracting data from Punching
Extracting data from Block
Extracting data from Idle


In [29]:
normalizeZValuesInCSV(CLASSES_LIST)

In [85]:
high_frame_videos = findFramesHigherThanX(CLASSES_LIST,100)
high_frame_videos

[]

In [84]:
deleteFramesHigherThanX(CLASSES_LIST,100)

In [73]:
highestFrameNumber, highestFrameVideo = findHighestFrameNumber(CLASSES_LIST)            

In [74]:
highestFrameNumber, highestFrameVideo

(216, 'Output\\Idle\\idle16.csv')

In [86]:
complete_df = createDataframe(CLASSES_LIST)

In [89]:
complete_df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,Kicking,"[[0.5277901887893677, 0.2767800390720367, 0.47...","[[0.5278607606887817, 0.2767897844314575, 0.48...","[[0.5278884172439575, 0.2766140401363373, 0.48...","[[0.5280408263206482, 0.2765996754169464, 0.48...","[[0.528099000453949, 0.2765790820121765, 0.482...","[[0.5281240940093994, 0.276489108800888, 0.482...","[[0.5281389951705933, 0.2764924168586731, 0.48...","[[0.5280817747116089, 0.2764426469802856, 0.48...","[[0.528153121471405, 0.2764979302883148, 0.483...",...,None,None,None,None,None,None,None,None,None,None
1,Kicking,"[[0.5225660800933838, 0.2767695784568786, 0.47...","[[0.5223726630210876, 0.2772463858127594, 0.49...","[[0.5223713517189026, 0.2779175043106079, 0.49...","[[0.5224747061729431, 0.2779271900653839, 0.49...","[[0.522502601146698, 0.2779575884342193, 0.493...","[[0.5225791335105896, 0.2779855132102966, 0.49...","[[0.5226031541824341, 0.278006374835968, 0.491...","[[0.5226078033447266, 0.278006762266159, 0.490...","[[0.5227125883102417, 0.2778693437576294, 0.49...",...,None,None,None,None,None,None,None,None,None,None
2,Kicking,"[[0.5231271982192993, 0.2770773470401764, 0.48...","[[0.5231088399887085, 0.2774262428283691, 0.49...","[[0.523113489151001, 0.2774796783924103, 0.494...","[[0.5232275724411011, 0.2782198786735534, 0.49...","[[0.5231906175613403, 0.2791962027549743, 0.48...","[[0.5230792760848999, 0.2797040343284607, 0.48...","[[0.5230745077133179, 0.2796937525272369, 0.48...","[[0.5231253504753113, 0.2796388566493988, 0.48...","[[0.5232998132705688, 0.2796262502670288, 0.48...",...,None,None,None,None,None,None,None,None,None,None
3,Kicking,"[[0.5240734815597534, 0.2751402556896209, 0.47...","[[0.5259157419204712, 0.2751370370388031, 0.46...","[[0.5265356302261353, 0.2752037346363067, 0.46...","[[0.527698278427124, 0.2752576470375061, 0.451...","[[0.5286362171173096, 0.2753035128116607, 0.44...","[[0.529110312461853, 0.2755385339260101, 0.453...","[[0.5292729139328003, 0.2755554020404815, 0.45...","[[0.5295383334159851, 0.2756929397583008, 0.45...","[[0.529764711856842, 0.2756904661655426, 0.466...",...,None,None,None,None,None,None,None,None,None,None
4,Kicking,"[[0.5169104337692261, 0.2728217840194702, 0.46...","[[0.5174950361251831, 0.2728206217288971, 0.47...","[[0.5181270241737366, 0.2725013792514801, 0.48...","[[0.5186636447906494, 0.272329181432724, 0.480...","[[0.5189972519874573, 0.2722953557968139, 0.47...","[[0.5192975997924805, 0.2722871899604797, 0.48...","[[0.5194026231765747, 0.2722907364368438, 0.47...","[[0.5196895599365234, 0.2722641229629516, 0.48...","[[0.519766092300415, 0.2721617817878723, 0.480...",...,None,None,None,None,None,None,None,None,None,None


In [91]:
labels = complete_df[0]
labels

0      Kicking
1      Kicking
2      Kicking
3      Kicking
4      Kicking
        ...   
169       Idle
170       Idle
171       Idle
172       Idle
173       Idle
Name: 0, Length: 174, dtype: object

In [92]:
features = complete_df.drop(0, axis=1)
features

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"[[0.5277901887893677, 0.2767800390720367, 0.47...","[[0.5278607606887817, 0.2767897844314575, 0.48...","[[0.5278884172439575, 0.2766140401363373, 0.48...","[[0.5280408263206482, 0.2765996754169464, 0.48...","[[0.528099000453949, 0.2765790820121765, 0.482...","[[0.5281240940093994, 0.276489108800888, 0.482...","[[0.5281389951705933, 0.2764924168586731, 0.48...","[[0.5280817747116089, 0.2764426469802856, 0.48...","[[0.528153121471405, 0.2764979302883148, 0.483...","[[0.5281022787094116, 0.2765122950077057, 0.48...",...,None,None,None,None,None,None,None,None,None,None
1,"[[0.5225660800933838, 0.2767695784568786, 0.47...","[[0.5223726630210876, 0.2772463858127594, 0.49...","[[0.5223713517189026, 0.2779175043106079, 0.49...","[[0.5224747061729431, 0.2779271900653839, 0.49...","[[0.522502601146698, 0.2779575884342193, 0.493...","[[0.5225791335105896, 0.2779855132102966, 0.49...","[[0.5226031541824341, 0.278006374835968, 0.491...","[[0.5226078033447266, 0.278006762266159, 0.490...","[[0.5227125883102417, 0.2778693437576294, 0.49...","[[0.5229853987693787, 0.2774359583854675, 0.49...",...,None,None,None,None,None,None,None,None,None,None
2,"[[0.5231271982192993, 0.2770773470401764, 0.48...","[[0.5231088399887085, 0.2774262428283691, 0.49...","[[0.523113489151001, 0.2774796783924103, 0.494...","[[0.5232275724411011, 0.2782198786735534, 0.49...","[[0.5231906175613403, 0.2791962027549743, 0.48...","[[0.5230792760848999, 0.2797040343284607, 0.48...","[[0.5230745077133179, 0.2796937525272369, 0.48...","[[0.5231253504753113, 0.2796388566493988, 0.48...","[[0.5232998132705688, 0.2796262502670288, 0.48...","[[0.5233860015869141, 0.2798021733760834, 0.48...",...,None,None,None,None,None,None,None,None,None,None
3,"[[0.5240734815597534, 0.2751402556896209, 0.47...","[[0.5259157419204712, 0.2751370370388031, 0.46...","[[0.5265356302261353, 0.2752037346363067, 0.46...","[[0.527698278427124, 0.2752576470375061, 0.451...","[[0.5286362171173096, 0.2753035128116607, 0.44...","[[0.529110312461853, 0.2755385339260101, 0.453...","[[0.5292729139328003, 0.2755554020404815, 0.45...","[[0.5295383334159851, 0.2756929397583008, 0.45...","[[0.529764711856842, 0.2756904661655426, 0.466...","[[0.5296931266784668, 0.2762711644172668, 0.47...",...,None,None,None,None,None,None,None,None,None,None
4,"[[0.5169104337692261, 0.2728217840194702, 0.46...","[[0.5174950361251831, 0.2728206217288971, 0.47...","[[0.5181270241737366, 0.2725013792514801, 0.48...","[[0.5186636447906494, 0.272329181432724, 0.480...","[[0.5189972519874573, 0.2722953557968139, 0.47...","[[0.5192975997924805, 0.2722871899604797, 0.48...","[[0.5194026231765747, 0.2722907364368438, 0.47...","[[0.5196895599365234, 0.2722641229629516, 0.48...","[[0.519766092300415, 0.2721617817878723, 0.480...","[[0.5197605490684509, 0.2719058990478515, 0.48...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,"[[0.500933051109314, 0.2933470904827118, 0.465...","[[0.5014461278915405, 0.2905421257019043, 0.47...","[[0.5047746896743774, 0.2905957400798797, 0.43...","[[0.507895290851593, 0.2908457219600677, 0.441...","[[0.508013129234314, 0.2908761203289032, 0.444...","[[0.5078443884849548, 0.2911202609539032, 0.44...","[[0.5083068013191223, 0.2913677990436554, 0.45...","[[0.5085481405258179, 0.2914840579032898, 0.45...","[[0.508915364742279, 0.291445642709732, 0.4572...","[[0.5098636150360107, 0.2913804054260254, 0.46...",...,None,None,None,None,None,None,None,None,None,None
170,"[[0.493659645318985, 0.2848674952983856, 0.450...","[[0.4934646189212799, 0.2854364812374115, 0.45...","[[0.4930198788642883, 0.2865460813045501, 0.45...","[[0.4931020736694336, 0.2892768085002899, 0.44...","[[0.4933908879756927, 0.2907748222351074, 0.43...","[[0.4935130178928375, 0.2907519042491913, 0.44...","[[0.4935278296470642, 0.2906800508499145, 0.44...","[[0.4934627115726471, 0.2907

In [ ]:
complete_df[complete_df.iloc[:,0] == 'Punching'][:5]

In [11]:
files = getAllCSV(CLASSES_LIST)

In [ ]:
df2 = pd.read_csv('output.csv',header=None,names=custom_headers)
arr = []
arr.extend(df2['z'].values)
arr

In [12]:
z_mean, z_std = calculateZParamsForNormalization(CLASSES_LIST,files)

In [13]:
z_min, z_max = calculateZMinMaxForNormalization(CLASSES_LIST,files)

In [34]:
z_mean, z_std
#(-0.10812381639618134, 0.44725407070063944)

(0.5545347004295016, 0.09994274934490296)

In [28]:
z_min, z_max
#(-1.6407425403594973, 1.754867672920227)

(0.0, 1.0)

In [ ]:
for class_name in CLASSES_LIST:        
    # Get list of csv files for each class
    files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))    
    for filename in files_list:
        if filename.endswith('.csv.csv'):
            folder_name = os.path.join(OUTPUT_DIR,class_name)
            # Construct the new filename by removing the extra .csv
            new_filename = os.path.join(folder_name, filename[:-4])
            # Rename the file
            os.rename(os.path.join(folder_name, filename), new_filename)
            print(f'Renamed {filename} to {filename[:-4]}')



In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
model = tf.keras.models.load_model('model.h5')

In [31]:
max_num_frames = 100  # maximum number of frames per action sequence
num_landmarks = 33    # number of landmarks per frame
num_features = 4      # number of features per landmark (x, y, z, visibility)
num_classes = len(CLASSES_LIST)       # number of action classes

In [32]:
def preprocess_frame(results):
    # Extract landmarks and preprocess
    landmarks = []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            landmarks.append([lm.x, lm.y, lm.z, lm.visibility])
    return landmarks

In [33]:
def predict_action(sequence):
    # Pad the sequence
    padded_sequence = pad_sequences([sequence], maxlen=max_num_frames, dtype='float32', padding='post', truncating='post', value=-1)
    padded_sequence = padded_sequence.reshape((padded_sequence.shape[0], padded_sequence.shape[1], num_landmarks * num_features))
    
    # Predict the action
    predictions = model.predict(padded_sequence)
    return np.argmax(predictions), np.max(predictions)

In [34]:
def normalize_z_values(landmarks):
    """
    Normalize the z-values of pose landmarks to a range of [0, 1].

    Parameters:
    landmarks (list): List of pose landmarks, each represented as [x, y, z, visibility].

    Returns:
    list: Normalized pose landmarks with z-values scaled to [0, 1].
    """
    z_values = [lm[2] for lm in landmarks]  # Extract all z-values

    if not z_values:
        return landmarks  # Return unchanged if no landmarks provided

    min_z = min(z_values)
    max_z = max(z_values)

    # Avoid division by zero if min_z equals max_z
    if max_z - min_z == 0:
        return landmarks

    # Normalize z-values to range [0, 1]
    normalized_landmarks = []
    for lm in landmarks:
        if max_z - min_z != 0:
            normalized_z = (lm[2] - min_z) / (max_z - min_z)
        else:
            normalized_z = lm[2]  # Handle edge case
        normalized_landmarks.append([lm[0], lm[1], normalized_z, lm[3]])  # Keep x, y, visibility unchanged

    return normalized_landmarks

In [43]:
# Start capturing video from webcam
cap = cv2.VideoCapture(0)

# Initialize a sequence to store landmarks
sequence = []
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Perform pose detection
        results = pose.process(image)

        # Convert the image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Draw the pose annotation on the image
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Extract landmarks and preprocess
            landmarks = preprocess_frame(results)
            normalized_landmarks = normalize_z_values(landmarks)
            if len(normalized_landmarks) > 0:
                sequence.append(normalized_landmarks)
            
            # If sequence is too long, remove the oldest frame
            if len(sequence) > max_num_frames:
                sequence.pop(0)
            
            # Make a prediction if sequence has enough frames
            if len(sequence) == max_num_frames:
                action_idx, confidence = predict_action(sequence)
                action_label = CLASSES_LIST[action_idx]
                
                # Display the predicted action on the frame
                cv2.putText(image, f'Action: {action_label} ({confidence:.2f})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display the resulting frame
        cv2.imshow('Webcam Action Recognition', image)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step


In [42]:
# # Start capturing video from webcam
# cap = cv2.VideoCapture(0)

# # Initialize Mediapipe Pose
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Convert the image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         # Perform pose detection
#         results = pose.process(image)

#         # Convert the image back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         # Draw the pose annotation on the image
#         if results.pose_landmarks:
#             mp_drawing.draw_landmarks(
#                 image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#         # Display the resulting frame
#         cv2.imshow('Webcam Pose Detection', image)
        
#         # Break the loop on 'q' key press
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

# # Release the capture
# cap.release()
# cv2.destroyAllWindows()